In [4]:
from utils import *

import numpy as np
import pickle
import sys

from keras.initializers import glorot_uniform
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras import backend as K

In [5]:
data_path = "data/parsed_text.txt"

try:
    with open(data_path, 'r', encoding='utf-8') as f:
        text = f.read()
except:
    text = get_raw_text()

In [8]:
cleaned_data_path = "data/cleaned_text.pickle"

try:
    with open(cleaned_data_path, 'rb') as p:
        cleaned_text = pickle.load(p)
except:
    cleaned_text = clean_and_tokenize_text(text)
    with open(cleaned_data_path, 'wb') as p:
        pickle.dump(cleaned_text, p)


In [9]:
word_to_index, index_to_word = create_dictionaries(cleaned_text)